####
# stategy of inference 
1. template 单独计算 metric--> comparing the the difference between them
2. all template together to caculate the metric
3. ensemble: vote OR mean() TO get metric

In [ ]:
export_model_batch_size = 32

In [ ]:
from typing import Tuple
import json
def test_data_prepare(path:str, type="ner")->Tuple[list,list]:
    "generate the test dataset from path in ncbi_blue dataset"
    try:
        with open(path,"r") as f:
            content = json.load(f)
    except:
        raise Exception(f"the file: {path} need to be #json# file type!")
    # 每个template 单独 计算 inference metric
    print("{} test dataset length is {}".format(path, len(content)))
    input_list = []
    label_list = []
    for i in content:   
        tmp = i.split("\t")    
        if len(tmp)<=1:
            raise Exception("input sentence spliting exception!")
        # require len(tmp)>=2 
        input = " ".join(tmp[:-1])
        if type=="ner":
            # 分割符号错误
            label = tmp[-1].split(",")
        elif type=="sentence_pairs":
            label = float(tmp[-1])
        input_list.append(input)
        # ,
        label_list.append(label)
    return input_list, label_list         

In [ ]:
def batch_data(inputs_data,batch_size=export_model_batch_size):
    "generator for batch test data"
    for start_idx in range(0, len(inputs_data), batch_size):
        excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs_data[excerpt]

In [ ]:
import numpy as np
import json
import requests
class SelfEncoder(json.JSONEncoder):  
    def default(self, obj):  
        if isinstance(obj, np.ndarray):  
            return obj.tolist()  
        elif isinstance(obj, np.floating):  
            return float(obj)  
        elif isinstance(obj, bytes):  
            return str(obj, encoding='utf-8');  
        return json.JSONEncoder.default(self, obj)
def ner_one_template(input_batch:list, model_name="blurb_t5"):
    '''input:batch_list []
        output:predicitons
    '''
    input = np.array(input_batch)
    input_data = {  
    "signature_name": "",  
    "instances":input}
    data = json.dumps(input_data, cls=SelfEncoder, indent=2)
    root_url = "http://10.100.45.205:8501"
    url = f"{root_url}/v1/models/{model_name}:predict"
    result = requests.post(url, data=data)
    tmp = eval(result.content)
    return_list = [i["outputs"].split(",") for i in tmp["predictions"]]
    return return_list 

In [ ]:
def ner_dataset_test(path:str):
    '''param: already prepared dataset_dir-->path
       func: collect and return all templates prediction result and label
    '''
    import os
    template_file_pth_list = []
    for _,_, files, in os.walk(path):
        for template_name in files:
            if template_name.endswith("json"):
                template_file_pth_list.append(template_name)
    all_templates_prediction= {}
    dataset_label = []

    for i in template_file_pth_list:
        tmp_prediction = []
        template_path = os.path.join(path, i)
        input, label = test_data_prepare(template_path)
        # get label
        if dataset_label==[]:
            dataset_label=label
    
        for input_batch in batch_data(input):
            tmp_prediction.extend(ner_one_template(input_batch))
        # collect all the templates applied test dataset's prediction of this ner dataset
        all_templates_prediction[i] = tmp_prediction
        print("template {} has been finished".format(i))
    # examine the prediction
    assert len(template_file_pth_list) == len(all_templates_prediction.keys())
    # 返回一个label
    return all_templates_prediction, label

In [ ]:
def metric_one_prompt(prediction, label):
    right_n = 0
    prediction_len = 0
    label_len = 0
    for predict_i,label_i in zip(prediction,label):
        for entity in predict_i:
            if entity in label_i:
                right_n+=1
        prediction_len+=len(predict_i)
        label_len += len(label_i)

    acc = float(right_n) / float(prediction_len)*100
    recall = float(right_n) / float(label_len)*100
    f1 = 2*(acc*recall)/(acc+recall)
    return {"f1":f1,"recall":recall, "acc":acc, "right_num":right_n, "label_entities":label_len, "prediction_len":prediction_len}

In [ ]:
def merge(predictions, label):
    all_prediction =[]
    for idx in range(len(label)):
        tmp = []
        for prompt in predictions.keys():
            tmp.extend(predictions[prompt][idx])
        all_prediction.append(list(set(tmp)))
    return all_prediction

def vote(predictions, label, threhold=2):
    from collections import Counter
    result_prediction = []
    for idx in range(len(label)):
        tmp = []
        for prompt in predictions.keys():
            tmp.extend(predictions[prompt][idx])
        tmp_c = Counter(tmp)
        if idx==0:
            print(tmp_c)
        ## vote strategy >half prompts num
        f_tmp = [i[0] for i in tmp_c.items() if i[1]>=threhold]
        if idx==0:
            print(f_tmp)
        result_prediction.append(list(f_tmp))
    return result_prediction


In [ ]:
THREHOLD = 2 # the threshold num of votes of templates 
def predict_result(predictions, label):
    predict_result = {}
    for i in predictions.keys():
        predict_result[i] = metric_one_prompt(predictions[i], label) 
    all_predict = merge(predictions, label) 
    vote_predict = vote(predictions, label, threhold=2)
    predict_result["add_all_templates"] = metric_one_prompt(all_predict, label)
    predict_result["vote_by_all_templates"] = metric_one_prompt(vote_predict, label)
    return predict_result,all_predict,vote_predict

### test start

In [ ]:
test_dir = "/raid/yiptmp/nlp_prepare_dataset/med0_dataset/blurb_test_dataset/{}"
test_dataset_path = {}
with open("ner_dataset_names", "r") as f:
    for i in f:
        test_dataset_path[i.strip()]=test_dir.format(i.strip())

In [ ]:
# 实体级别的测试指标

In [ ]:

import json
import os

model_name = "t5_base"
result_dir = "/raid/zyftest/project/Med0/t5_multitasks_finetune/eval_metric/test_result"
for dataset_name,path in test_dataset_path.items():
    predictions,label = ner_dataset_test(path)
    tmp,all_tem_prediction,vote_tem_prediction = predict_result(predictions, label)
    # save the prediction
    predictions["all_templates_output"] = all_tem_prediction
    predictions["vote_templates_output"] = vote_tem_prediction
    content = {"predictions":predictions, "label":label}
    with open(path+"/"+f"{model_name}_output_result","w") as result_w:
        json.dump(content, result_w, indent=2, ensure_ascii=False)
        
    
    save_dir_path = os.path.join(result_dir, model_name)
    save_pth = os.path.join(save_dir_path, dataset_name)
    if not os.path.exists(save_dir_path):
        os.makedirs(save_dir_path)
    
    with open(save_pth, "w") as f:
        json.dump(tmp, f, indent=2, ensure_ascii=False)    